In [12]:
from langchain_community.document_loaders import PDFMinerLoader

In [24]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
loader=PDFMinerLoader(file_path="sample_book.pdf")
loader

In [3]:
doc = loader.load()

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(doc)

In [5]:
documents[0]

Document(metadata={'source': 'sample_book.pdf'}, page_content="MANTRESWARA'S\n\nPHALA DEEPIKA\n\nEnglish Translation, Commentary\n\nand annotation by Dr. G. S. Kapoor\n\n\x0cPreface\n\nShri   Mantreswara's   'Phaladeeplka'   holds   a   most   distinguished   position \n\namongst   the   great   ancient   astrological   classics   alongwith   Brihat   Parasara   Hora \n\nSastra, Varahmihira's Brihat Jataka and Vaidya Natha's Jataka Parijata. It deals in a \n\nmost   comprehensive   and   authoritative   manner   with   almost   all   the   astrological \n\naspects of human life. There can be said to be nothing that this great work has \n\nomitted from its purview.\n\nSome of the important subjects dealt with in a lucid manner in the book are: \n\ndescription   and   characteristic   features   of   the   various   signs,   planets   and   bhavas \n\n(houses), the various kinds of strengths of planets and their awasthas with their \n\neffects, sources of livelihood, yogas or planetary c

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [7]:
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

In [17]:
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [18]:
vectorstoredb

In [ ]:
vectorstoredb.save_local("faiss_index")

In [9]:
vectorstoredb_local = FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)

In [10]:
query="Saturn in 6th house"
result=vectorstoredb_local.similarity_search(query)
result[0].page_content

'indolent and unhappy.\n\nIf Saturn be in the 4th house at birth, the native will be bereft of house, \n\nrespect and mother. He will be sickly in his childhood. He will also be unhappy.\n\n99\n\n\x0cIf Saturn occupied the 5th house the person concerned will be evil minded \n\nand wicked.  He  will  be  bereft  of  learning,  children,   wealth  and  happiness.  He \n\nroams about aimlessly.\n\nShould Saturn occupy the 6th house at birth, the person concerned will be \n\nglutton, wealthy and will vanquish his enemies. He will also be arrogant.\n\nWhen   at   birth   Saturn   is   posited   in   the   7th   house,   the   native   develops \n\nrelations  with  women of questionable  character. He will  be poverty  stricken, a \n\nwanderer and unhappy.\n\nSaturn   occupying   the   8th   house   at   birth   will   make   the   native   unclean \n\nafflicted  with   piles,  devoid  of  wealth,  of  cruel   disposition,  hungry  and  will  be \n\navoided by his friends.'

In [11]:
result

[Document(metadata={'source': 'sample_book.pdf'}, page_content='indolent and unhappy.\n\nIf Saturn be in the 4th house at birth, the native will be bereft of house, \n\nrespect and mother. He will be sickly in his childhood. He will also be unhappy.\n\n99\n\n\x0cIf Saturn occupied the 5th house the person concerned will be evil minded \n\nand wicked.  He  will  be  bereft  of  learning,  children,   wealth  and  happiness.  He \n\nroams about aimlessly.\n\nShould Saturn occupy the 6th house at birth, the person concerned will be \n\nglutton, wealthy and will vanquish his enemies. He will also be arrogant.\n\nWhen   at   birth   Saturn   is   posited   in   the   7th   house,   the   native   develops \n\nrelations  with  women of questionable  character. He will  be poverty  stricken, a \n\nwanderer and unhappy.\n\nSaturn   occupying   the   8th   house   at   birth   will   make   the   native   unclean \n\nafflicted  with   piles,  devoid  of  wealth,  of  cruel   disposition,  hungr

In [14]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10f4b1330>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10f4b1ff0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [15]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10f4b1330>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10f4b1ff0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [16]:
from langchain_core.documents import Document
document_chain.invoke({
    "context":[Document(page_content="there are some important concepts in astrology")]
})

'The context states that there are important concepts in astrology.  \n'

In [14]:
vectorstoredb

In [18]:
retriever=vectorstoredb_local.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [19]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10dafdcf0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | ChatGro

In [21]:
response=retrieval_chain.invoke({"input":"venus in 12th house"})
response['answer']

'This text describes the astrological meanings of Venus in different houses. \n\nIt states that if Venus is in the 2nd house at birth, the native will be a poet and enjoy riches of various kinds. \n'

In [22]:
response

{'input': 'venus in 12th house',
 'context': [Document(metadata={'source': 'sample_book.pdf'}, page_content='fearless. He gets few children; but enjoys the comforts of conveyances.\n\nWhen Jupiter  occupies the  12th  house,  the  native  will  earn  animosity  ol \n\nothers,   and   will   himself   use   abusive   language.   He   will   be   childless   ,   sinful, \n\nindolent and a person who serves others.\n\nVENUS\n\n17-19.   If  Venus   be   posited   in   the   lagna   at   birth,   the   native   will   have   a \n\nbeautiful and attractive body, and will be happy and longlived.\n\nNotes—Venus is the benefic No. 2. He can never be as beneficial as Jupiter \n\nin the Lagna because he possesses one lull aspect only and that will fall on the 7th \n\nhouse only.\n\nShould Venus occupy the 2nd house at birth, the native will be a poet and \n\nwill enjoy riches of various kinds.\n\nIf Venus be In the 3rd house at birth, the native will be miserly, unpopular, \n\ndevoid of wealth, h

In [23]:
response['context']

[Document(metadata={'source': 'sample_book.pdf'}, page_content='fearless. He gets few children; but enjoys the comforts of conveyances.\n\nWhen Jupiter  occupies the  12th  house,  the  native  will  earn  animosity  ol \n\nothers,   and   will   himself   use   abusive   language.   He   will   be   childless   ,   sinful, \n\nindolent and a person who serves others.\n\nVENUS\n\n17-19.   If  Venus   be   posited   in   the   lagna   at   birth,   the   native   will   have   a \n\nbeautiful and attractive body, and will be happy and longlived.\n\nNotes—Venus is the benefic No. 2. He can never be as beneficial as Jupiter \n\nin the Lagna because he possesses one lull aspect only and that will fall on the 7th \n\nhouse only.\n\nShould Venus occupy the 2nd house at birth, the native will be a poet and \n\nwill enjoy riches of various kinds.\n\nIf Venus be In the 3rd house at birth, the native will be miserly, unpopular, \n\ndevoid of wealth, happiness and wife.\n\nWith Venus occupying th

In [8]:
# prompt=ChatPromptTemplate.from_messages(
#     [
#         ("system","You are a helpful assistant. Please respond to the question asked"),
#         ("user","Question:{question}")
#     ]
# )
# llm=Ollama(model="llama3")
# output_parser=StrOutputParser()
# chain=prompt|llm|output_parser
# chain.invoke({"question":"what is the effect of venus in 12th house?"})